In [83]:
import json
import os
import sys

import joblib

# Добавляем корневую директорию проекта в sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../../")))

# Импортируем metrics из utils
from utils import metrics as mtc
from utils import preprocess_functions as ppf

In [84]:
import numpy as np
import pandas as pd

# Готовим дата-сеты (предобработка)

In [85]:
reference = pd.read_csv("../../data/preprocessed/general/reference.csv")
train = pd.read_csv("../../data/splited/train.csv")
valid = pd.read_csv("../../data/splited/valid.csv")

In [86]:
reference["processed_name"] = reference.name.apply(
    ppf.simple_preprocess_text
).str.lower()
reference.processed_name = reference.processed_name.apply(ppf.replace_numbers_with_text)
reference.head(1)

,id,name,region,processed_name
0,69,Кировска,мурманская область,кировска


In [87]:
train["processed_name"] = train.name.apply(ppf.simple_preprocess_text).str.lower()
train.processed_name = train.processed_name.apply(ppf.replace_numbers_with_text)
train.head(1)

,name,school_id,processed_name
0,"Москва, ГБУ МАФКК школа ""Мечта""",102,москва гбу мафкк школа мечта


In [88]:
valid["processed_name"] = valid.name.apply(ppf.simple_preprocess_text).str.lower()
valid.processed_name = valid.processed_name.apply(ppf.replace_numbers_with_text)
valid.head(1)

,name,school_id,processed_name
0,"Санкт-Петербург, СПб СШОР",198,санкт петербург спб сшор


# Данные для обучения

In [89]:
merged_train = train.join(
    reference.set_index("id"), on="school_id", how="left", rsuffix="_reference"
)
merged_train = merged_train[~merged_train.name_reference.isna()]
merged_train = merged_train[~merged_train.processed_name.isna()]
merged_train = merged_train[["processed_name", "processed_name_reference"]]
merged_train["target"] = 1

In [90]:
merged_train.head()

,processed_name,processed_name_reference,target
0,москва гбу мафкк школа мечта,мечта,1
1,калужская область сш космос,космос,1
2,мо семь округ ооо династия,династия,1
3,анфсо скфк спартак,спартак,1
4,челябинская область ип дмитриев,команда дмитриева,1


In [91]:
temp_df = merged_train.copy()
train_classif = merged_train.copy()

for i in range(0, 5, 1):
    temp_df = temp_df.copy()

    # Параметр для сдвига
    shift_amount = 15  # Положительное значение сдвигает вниз, отрицательное - вверх

    # Сдвиг второго столбца
    temp_df["processed_name_reference"] = temp_df["processed_name_reference"].shift(
        shift_amount
    )
    temp_df["target"] = 0

    # Новый датафрейм с исходным и сдвинутым столбцами
    temp_df = temp_df.dropna()

    train_classif = pd.concat([train_classif, temp_df])

train_classif.reset_index(drop=True, inplace=True)

In [92]:
train_classif.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3075 entries, 0 to 3074
Data columns (total 3 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   processed_name            3075 non-null   object
 1   processed_name_reference  3075 non-null   object
 2   target                    3075 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 72.2+ KB


In [93]:
# Объединение текстов для векторизации
train_classif["combined"] = (
    train_classif["processed_name"] + " " + train_classif["processed_name_reference"]
)

In [94]:
# Разделение на признаки и целевую переменную
X = train_classif["combined"]
y = train_classif["target"]

In [95]:
from sklearn.model_selection import train_test_split

# Разделение данных на обучающую и тестовую выборки

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# CATBOOST

In [96]:
from catboost import CatBoostClassifier
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report

In [97]:
# Создание пайплайна для векторизации, балансировки классов и обучения модели логистической регрессии
pipeline = ImbPipeline(
    [
        ("tfidf", TfidfVectorizer()),
        ("smote", SMOTE(sampling_strategy="auto", k_neighbors=5, random_state=42)),
        (
            "clf",
            CatBoostClassifier(
                iterations=1000, verbose=100, auto_class_weights="SqrtBalanced"
            ),
        ),
    ]
)

In [98]:
# Обучение модели
pipeline.fit(X_train, y_train)

Learning rate set to 0.018701
0:	learn: 0.6895443	total: 19.5ms	remaining: 19.5s


100:	learn: 0.4971405	total: 2.17s	remaining: 19.3s
200:	learn: 0.4183960	total: 3.75s	remaining: 14.9s
300:	learn: 0.3660494	total: 5.09s	remaining: 11.8s
400:	learn: 0.3247048	total: 6.26s	remaining: 9.36s
500:	learn: 0.2868607	total: 7.71s	remaining: 7.67s
600:	learn: 0.2503484	total: 8.86s	remaining: 5.88s
700:	learn: 0.2225492	total: 10.1s	remaining: 4.29s
800:	learn: 0.2002023	total: 11.3s	remaining: 2.8s
900:	learn: 0.1819425	total: 12.5s	remaining: 1.37s
999:	learn: 0.1668061	total: 13.6s	remaining: 0us


Pipeline(steps=[('tfidf', TfidfVectorizer()), ('smote', SMOTE(random_state=42)),
                ('clf',
                 <catboost.core.CatBoostClassifier object at 0x0000022FD5071E20>)])

In [99]:
# Оценка модели
y_pred_test = pipeline.predict(X_test)

In [100]:
print(classification_report(y_test, y_pred_test))

              precision    recall  f1-score   support

           0       0.94      0.93      0.94       505
           1       0.70      0.75      0.72       110

    accuracy                           0.90       615
   macro avg       0.82      0.84      0.83       615
weighted avg       0.90      0.90      0.90       615



# Данные для валидации

In [101]:
merged_valid = valid.join(
    reference.set_index("id"), on="school_id", how="left", rsuffix="_reference"
)
merged_valid = merged_valid[~merged_valid.name_reference.isna()]
merged_valid = merged_valid[~merged_valid.processed_name.isna()]
merged_valid = merged_valid[["processed_name", "processed_name_reference"]]

In [102]:
merged_valid.head()

,processed_name,processed_name_reference
0,санкт петербург спб сшор,спб гбу до сшор академия фкк
1,липецкая область мауо до сш одиннадцать,сш одиннадцать
2,москва ано лига фигурного катания,ано лига фигурного катания
3,кфк динамо санкт петербург,нп кфк динамо санкт петербург
4,ленинградская область мбау всшор,всшор


In [103]:
valid_pos_classif = pd.DataFrame()

# Объединение текстов для векторизации
valid_pos_classif["left"] = merged_valid["processed_name"]
valid_pos_classif["right"] = merged_valid["processed_name_reference"]

valid_pos_classif["combined"] = (
    valid_pos_classif["left"] + " " + valid_pos_classif["right"]
)
valid_pos_classif = valid_pos_classif.reset_index(drop=True)

In [104]:
# Оценка модели
y_pred_valid_pos = pipeline.predict(valid_pos_classif["combined"])

In [105]:
y_pred_valid_pos

array([1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0,
       0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1], dtype=int64)

In [106]:
sum(y_pred_valid_pos) / len(y_pred_valid_pos)

0.8131868131868132

In [107]:
valid_neg_classif = pd.DataFrame()

# Объединение текстов для векторизации
valid_neg_classif["left"] = merged_valid["processed_name"]
valid_neg_classif["right"] = merged_valid["processed_name_reference"].shift(5)

valid_neg_classif["combined"] = (
    valid_neg_classif["left"] + " " + valid_neg_classif["right"]
)

valid_neg_classif = valid_neg_classif[~valid_neg_classif.combined.isna()]

valid_neg_classif = valid_neg_classif.reset_index(drop=True)

In [108]:
# Оценка модели
y_pred_valid_neg = pipeline.predict(valid_neg_classif["combined"])

In [109]:
(len(y_pred_valid_neg) - sum(y_pred_valid_neg)) / len(y_pred_valid_neg)

0.9096045197740112

# Изучение остатков

In [110]:
ids_pos_wrong = np.where(y_pred_valid_pos == 0)[0]
len(ids_pos_wrong.tolist())

34

In [111]:
ids_pos_wrong

array([  2,   7,  16,  27,  33,  35,  36,  38,  43,  44,  47,  53,  54,
        56,  57,  59,  74,  75,  76,  78,  81,  94, 101, 121, 123, 131,
       133, 137, 156, 162, 164, 168, 171, 178], dtype=int64)

In [112]:
valid_pos_classif.loc[ids_pos_wrong.tolist(), ["left", "right"]]

,left,right
2,москва ано лига фигурного катания,ано лига фигурного катания
7,республика мордовия гбу до сш академия фкк,академия фкк
16,когау до сш дымка,дымка
27,нижегородская область мау до спортивная школа ...,сш один
33,мо правобережный ооо хк ска,хоккейный клуб ска
35,ано фсо кфкнк звезда,звезда
36,ск фкк олимпия свердловская область,нлфк
38,владимирская область вро ого вфсо динамо,динамо
43,москва ано просинхро,просинхро
44,мбоудо иркутск дюсш семь иркутская обл,дюсш семь


In [113]:
ids_neg_wrong = np.where(y_pred_valid_neg == 1)[0]
ids_neg_wrong.tolist()

[7, 10, 16, 26, 62, 74, 75, 98, 117, 119, 141, 143, 147, 150, 168, 176]

In [114]:
valid_neg_classif.loc[ids_neg_wrong.tolist(), ["left", "right"]]

,left,right
7,локомотив,академия фкк
10,гбу дюц московского района цфксиз,цфксиз фрунзенского района
16,кфк титул,дымка
26,шсфк маска,цфксиз василеостровского района
62,ямало ненецкий ао гау до янао сш ямал,авиатор
74,москва гбу до мафкк школа медведково,медведково
75,красноярский край кгаудо сш центр по лвс,сшор по фккишт
98,костромская область гбу до ко сшор им голубева,сшор два
117,спб гбу цфкиз фрунзенского на санкт петербург,спб гбу до сшор академия фкк
119,янао сш арктур,титул


# Применение модели

In [115]:
reference_id = reference["id"].to_numpy(dtype="int").flatten()
reference_name = reference["processed_name"].to_numpy(dtype="str").flatten()
reference_region = reference["region"].to_numpy(dtype="str").flatten()

In [116]:
joblib.dump(reference_id, "../../resources/exp36/reference_id.joblib")
joblib.dump(reference_name, "../../resources/exp36/reference_name.joblib")
joblib.dump(reference_region, "../../resources/exp36/reference_region.joblib")

['../../resources/exp36/reference_region.joblib']

In [117]:
x_valid = valid["processed_name"].to_numpy(dtype="str").flatten()
y_valid = valid["school_id"].to_numpy(dtype="int").flatten()
region_valid = np.full(train.shape[0], "unknown")

In [118]:
y_pred_final = []

for x_test in x_valid:
    x_test_process = np.char.add(x_test + " ", reference_name)

    # Оценка модели
    y_pred_test = pipeline.predict_proba(x_test_process)[:, 1]

    # Найти индексы, отсортированные по убыванию значений
    sorted_indices = np.argsort(y_pred_test)[::-1]

    # Выбрать первые пять индексов
    top_five_indices = sorted_indices[:5]

    # Получить значения этих элементов
    top_five_values = y_pred_test[top_five_indices]

    # Создать массив с кортежами (id элемента, значение)
    result = np.array(
        [
            (reference_id[index], value)
            for index, value in zip(top_five_indices, top_five_values)
        ],
        dtype=[("index", int), ("value", float)],
    )

    y_pred_final.append(result)

# Преобразовать итоговый список в массив numpy
y_pred_final = np.array(y_pred_final)

In [119]:
y_pred_final

array([[( 198, 0.60771823), ( 262, 0.54403354), (  47, 0.49688714),
        (  46, 0.49688714), ( 234, 0.49505434)],
       [( 184, 0.90334977), ( 285, 0.87838567), ( 248, 0.81063138),
        (1845, 0.80356187), ( 286, 0.78103276)],
       [( 285, 0.79802693), ( 231, 0.68972041), ( 107, 0.63428598),
        (  10, 0.62834289), (  21, 0.59714714)],
       [( 277, 0.82416086), (  44, 0.68035699), ( 224, 0.60458842),
        ( 610, 0.57992394), ( 221, 0.57992394)],
       [( 248, 0.63385111), ( 294, 0.60717383), ( 285, 0.59339169),
        (  35, 0.57697832), ( 282, 0.56919969)],
       [( 121, 0.95008662), ( 120, 0.95008662), ( 119, 0.95008662),
        ( 118, 0.95008662), ( 122, 0.95008662)],
       [( 110, 0.96202663), ( 285, 0.93990789), (1845, 0.91727951),
        ( 282, 0.91458886), ( 248, 0.91138785)],
       [( 285, 0.83026502), (  10, 0.80268685), ( 294, 0.7367193 ),
        ( 231, 0.72400738), ( 248, 0.69327561)],
       [( 282, 0.77406859), ( 128, 0.76629526), ( 231, 0.5328271

In [120]:
y_pred_valid_pos

array([1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0,
       0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1], dtype=int64)

In [121]:
y_valid

array([ 198,  184,  293,  277,   35, 1843,  110,   11,  128,   25,  257,
        222,   89,  200,   19,  256,   50,   85,   83,  212,  223,  223,
        254,   78,   35,  174,  254,  183,   60,  265,   20,   93,  139,
        131,  156, 1851,  111,   61,  112,   41,   46,   24,  246,  212,
        142,  136,   58,   51,   48,   81,  251,   90,  203,  111,  128,
         46,   61,   87,  259,  135,  264,  264,  154,   37,    3,  244,
        159,  126,  227,  267,  102,  157,  169,   90,  256,  244,  101,
        201,  172,   90,   81,  101,  246,  259,  195,  123,  102,  244,
         37,   24,  227,  201,  101,   54,  256,  121,  303,  182,  152,
        179,  209,  156,  118,  293,  249,   40,  195,   50,  143,   82,
        116,  192,  144,   92,  181,   97,   85,  287,  173,  198,  205,
        223,  198,   61,  257,  244,   19,  135,   50,   82,  143,   21,
        212,  124,   31,  256,   28,  166,   37,  185,    9,   35,  119,
        248,  264,  304,  215,  254,   65,  166,  1

In [122]:
y_pred_final_1 = [k[0][0] for k in y_pred_final]

In [123]:
df = pd.DataFrame(
    [(y_valid[i], y_pred_final_1[i], y_pred_valid_pos[i]) for i in range(0, 181)],
    columns=["y_valid", "y_pred_final_1", "y_pred_valid_pos"],
)
df

,y_valid,y_pred_final_1,y_pred_valid_pos
0,198,198,1
1,184,184,1
2,293,285,0
3,277,277,1
4,35,248,1
...,...,...,...
176,92,282,1
177,167,285,1
178,90,296,0
179,176,142,1


In [124]:
metrics = mtc.calculate_metrics(y_valid, y_pred_final, [])
metrics

{'Accuracy@1': 0.332,
 'Accuracy@3': 0.582,
 'Accuracy@5': 0.652,
 'auto_error_rate': 0.668,
 'manual_processing_rate': 0.0,
 'general_error': 0.668}

In [125]:
# Сохраняем словарь в файл JSON
with open("../../evaluations/exp36.json", "w", encoding="utf-8") as f:
    json.dump(metrics, f, ensure_ascii=False, indent=4)

In [126]:
x_test = x_valid[:1]
x_test

array(['санкт петербург спб сшор'], dtype='<U59')

In [127]:
x_test_process = np.char.add(x_test[0] + " ", reference_name)

In [128]:
# Оценка модели
y_pred_test = pipeline.predict_proba(x_test_process)[:, 1]

In [129]:
# Найти индексы, отсортированные по убыванию значений
sorted_indices = np.argsort(y_pred_test)[::-1]

# Выбрать первые пять индексов
top_five_indices = sorted_indices[:5]

# Получить значения этих элементов
top_five_values = y_pred_test[top_five_indices]

# Создать массив с кортежами (id элемента, значение)
result = np.array(
    [
        (reference_id[index], value)
        for index, value in zip(top_five_indices, top_five_values)
    ],
    dtype=[("index", int), ("value", float)],
)

result

array([(198, 0.60771823), (262, 0.54403354), ( 47, 0.49688714),
       ( 46, 0.49688714), (234, 0.49505434)],
      dtype=[('index', '<i4'), ('value', '<f8')])